In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
#text to numeric
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
#warnings
import warnings
warnings.filterwarnings('ignore')

In [12]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

print the stopword in english

In [13]:
print('stopwords:',stopwords.words('english'))

stopwords: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so'

#load dataset from kaggle using api

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

importing tweeter sentiment dataset

In [6]:
#api to load dataset
#!/bin/bash
!kaggle datasets download kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 80% 65.0M/80.9M [00:00<00:00, 170MB/s]
100% 80.9M/80.9M [00:00<00:00, 162MB/s]


#unzip the file

#load the dataset after unzipping

In [7]:
from zipfile import ZipFile
dataset='/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('finish')

finish


In [17]:
df=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',header=None,encoding='ISO-8859-1')

In [18]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [21]:
#check the shape of dataset
df.shape

(1600000, 6)

In [19]:
#rename the column name
column_name=['target','ids','date','flag','user','text']
df.columns=column_name

In [20]:
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [22]:
#check the missing values in dataset
df.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [23]:
#check the duplicated column
df.duplicated().sum()

0

target:- the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

In [24]:
#check the target column values count
df.value_counts('target')

,count
target,
0,800000
4,800000


In [25]:
#convert the 4 label in target column in to 1
df['target']=df['target'].replace(4,1)

In [28]:
df['target'].value_counts()

,count
target,
0,800000
1,800000


where 1=Positive and 0=Negative

#stemming
-it is process of reducing a word to its root word

In [29]:
port_stem=PorterStemmer()

In [31]:
#create steamming function
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  #lower letter the every word in content
  stemmed_content=stemmed_content.lower()
  #split the sentence then consider the word in stemmed_content
  stemmed_content=stemmed_content.split()
  #remove stopword from sentence in text
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  #join the content word in to stemmed_content one by one
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [33]:
#create column steammed content
df['stemmed_content']=df['text'].apply(stemming)  # it take more then 50 minutes to run

In [34]:
#check the dataset with new column called steammed column
df.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [35]:
df['stemmed_content'].head()

,stemmed_content
0,switchfoot http twitpic com zl awww bummer sho...
1,upset updat facebook text might cri result sch...
2,kenichan dive mani time ball manag save rest g...
3,whole bodi feel itchi like fire
4,nationwideclass behav mad see


In [36]:
#we don't need the id and other columns only need to target column and stemmed_content column
X=df['stemmed_content'].values
y=df['target'].values

#split the datset in to training data and test dataset

In [37]:
#for equal 0 and 1 distribution bewteen train and test data use (Stratify=Y)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)

In [38]:
#check the shape
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [39]:
#check the shape of test
print(y.shape,y_train.shape,y_test.shape)

(1600000,) (1280000,) (320000,)


In [40]:
#convert the textual data to numerical data
vectorizer=TfidfVectorizer()

X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [44]:
#check the data convert to numerical
print(X_train)

  (0, 307108)	0.46206048815324474
  (0, 239679)	0.15130037108228483
  (0, 146067)	0.12929728405657018
  (0, 175252)	0.224070805470346
  (0, 128605)	0.22108856600702773
  (0, 4832)	0.317074267861159
  (0, 124524)	0.18318401951949756
  (0, 205794)	0.24140229063801746
  (0, 454381)	0.20169626473577715
  (0, 286478)	0.16123218610004272
  (0, 406297)	0.2978221095272138
  (0, 220296)	0.43015677907624866
  (0, 388138)	0.20555120011808467
  (0, 154767)	0.26976607043258233
  (1, 445870)	0.6361096685891185
  (1, 161801)	0.5778049407933611
  (1, 124611)	0.5113765148324884
  (2, 125319)	0.6383069130836649
  (2, 349409)	0.22232944888223494
  (2, 444761)	0.30331529032956345
  (2, 358186)	0.19837942712286838
  (2, 267649)	0.19309660201644555
  (2, 12436)	0.2529872032123258
  (2, 453420)	0.2347069337186747
  (2, 312657)	0.3154702974657607
  :	:
  (1279997, 124611)	0.2537781914144255
  (1279997, 301683)	0.1908678391932645
  (1279997, 168646)	0.20490659397970187
  (1279997, 156392)	0.23570311036994007
 

In [47]:
print(X_test)

  (0, 28874)	0.1778395103911245
  (0, 43712)	0.23562815302828183
  (0, 78636)	0.5158100011206617
  (0, 96399)	0.255967788489452
  (0, 97585)	0.4019235611854435
  (0, 129417)	0.25650960779862714
  (0, 189057)	0.31324918577405797
  (0, 240451)	0.15341308097014625
  (0, 334643)	0.14719329779308424
  (0, 335577)	0.22602158147814247
  (0, 384697)	0.3281164007446601
  (0, 435956)	0.14183025329879742
  (0, 453357)	0.1781708363247895
  (1, 35118)	0.3128685670821343
  (1, 36669)	0.2729958846742327
  (1, 39445)	0.16867093960211466
  (1, 57115)	0.19043301054662504
  (1, 74274)	0.21148120876702692
  (1, 78790)	0.13386322067407883
  (1, 170374)	0.17525273735418329
  (1, 171245)	0.12468774856570086
  (1, 183279)	0.24586158827112847
  (1, 233854)	0.3852709938491561
  (1, 240223)	0.1674195650536303
  (1, 301683)	0.13212235134015302
  :	:
  (319997, 135536)	0.21809964977532384
  (319997, 146067)	0.31485892246365776
  (319997, 169469)	0.1945044482645627
  (319997, 225096)	0.2933179603743362
  (319997, 3

#build the model

In [50]:
#classification model to train the dataset
model=LogisticRegression(max_iter=1000)

In [51]:
#model fiting
model.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [52]:
#model evaluation on accuracy score on traing data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,y_train)

In [53]:
#print accuracy sore on traing data
print('accuracy score on training data:',training_data_accuracy)

accuracy score on training data: 0.7999984375


In [54]:
#model evaluation on accuracy score on test  data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,y_test)

In [55]:
#print accuracy sore on traing data
print('accuracy score on test data:',test_data_accuracy)

accuracy score on test data: 0.776796875


-***training data accuracy and test data acuracy very close to each other there are no sign of overfitting/underfitting***

#Save the model

In [56]:
import pickle
pickle.dump(model,open('sentiment_model.sav','wb'))

#load the saved model file

In [57]:
load_model=pickle.load(open('/content/sentiment_model.sav','rb'))

In [64]:
X_new=X_test[150]
prediction=load_model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print('The News in tweet is Negative')
else:
  print('The News in tweet is Positive')

[1]
The News in tweet is Positive


In [69]:
X_new=X_test[30]
prediction=load_model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print('The News in tweet is Negative')
else:
  print('The News in tweet is Positive')

[0]
The News in tweet is Negative


***Model working perfectly fine it predict both positive and negative tweets***